In [3]:
import importlib, os, pandas as pd



# Notebook and script are in same directory

import transform_bronze_gkg_aml as aml

importlib.reload(aml)

print('Loaded module from:', aml.__file__)


Loaded module from: c:\Users\gonca\jupyter\gdelt_data_vault\workspace\scripts\transform_bronze_gkg_aml.py


In [4]:
scripts_dir_candidates = [

    os.path.join(os.getcwd(), 'workspace', 'scripts'),

    os.path.join(os.getcwd(), 'scripts')

]

scripts_dir = next((d for d in scripts_dir_candidates if os.path.exists(os.path.join(d, 'transform_bronze_gkg_aml.py'))), None)

assert scripts_dir, 'Could not find transform_bronze_gkg_aml.py'

if scripts_dir not in sys.path:

    sys.path.insert(0, scripts_dir)

print('Using scripts_dir:', scripts_dir)

AssertionError: Could not find transform_bronze_gkg_aml.py

In [32]:
def find_scripts_dir():

    """Locate the directory containing transform_bronze_gkg_aml.py by walking up the cwd tree.

    Works inside Jupyter where __file__ is undefined."""

    target_file = 'transform_bronze_gkg_aml.py'

    cwd = Path.cwd().resolve()

    # Search common patterns from each ancestor

    for base in [cwd] + list(cwd.parents):

        # Pattern A: base/workspace/scripts

        cand = base / 'workspace' / 'scripts'

        if (cand / target_file).exists():

            return cand

        # Pattern B: base/scripts

        cand2 = base / 'scripts'

        if (cand2 / target_file).exists():

            return cand2

    raise FileNotFoundError(f'Could not locate {target_file} starting from {cwd}')



scripts_dir = find_scripts_dir()

workspace_dir = scripts_dir.parent  # workspace

repo_root = workspace_dir.parent    # repo root



print('Detected scripts_dir:', scripts_dir)

print('Detected workspace_dir:', workspace_dir)

print('Detected repo_root:', repo_root)



if str(scripts_dir) not in sys.path:

    sys.path.insert(0, str(scripts_dir))

print('sys.path head:', sys.path[:3])



import transform_bronze_gkg_aml as aml

importlib.reload(aml)

Detected scripts_dir: /workspace/workspace/scripts
Detected workspace_dir: /workspace/workspace
Detected repo_root: /workspace
sys.path head: ['/workspace/workspace/scripts', '/workspace/workspace/scripts/workspace/scripts', '/usr/lib/python39.zip']


TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'

In [33]:
input_dir, output_dir = aml._default_paths()

print('Input dir:', input_dir)

print('Output dir (auto):', output_dir)

print('Exists output?', os.path.isdir(output_dir))

Input dir: /workspace/workspace/data/raw
Output dir (auto): /workspace/workspace/data/raw/bronze
Exists output? True


In [34]:
sample_file = '20230105060000.gkg.csv'

sample_path = os.path.join(input_dir, sample_file)

print('Sample file path:', sample_path, 'Exists?', os.path.exists(sample_path))


Sample file path: /workspace/workspace/data/raw/20230105060000.gkg.csv Exists? True


In [35]:
# 3. Manually inspect first few lines + theme column distribution

df_head = pd.read_csv(sample_path, sep='\t', header=None, dtype=str, nrows=25, encoding='utf-8')

print('Head shape:', df_head.shape)

Head shape: (25, 27)


In [36]:
if 8 in df_head.columns:

    print('Theme col examples:')

    print(df_head[8].head(10))

else:

    print('Column 8 missing in sample head!')

Theme col examples:
0                                                  NaN
1    GENERAL_GOVERNMENT,1972;EPU_POLICY_GOVERNMENT,...
2    CRISISLEX_CRISISLEXREC,2260;TAX_ECON_FREETRADE...
3    CRISISLEX_C07_SAFETY,2124;EPU_CATS_REGULATION,...
4    RURAL,3862;SHORTAGE,3540;WATER_SECURITY,3102;W...
5    TAX_FNCACT_HELPERS,3851;TAX_FNCACT_HELPERS,488...
6    TAX_FNCACT_METEOROLOGIST,448;NATURAL_DISASTER_...
7    DELAY,6493;UNGP_FORESTS_RIVERS_OCEANS,2710;UNG...
8                      GENERAL_HEALTH,156;MEDICAL,156;
9                                                  NaN
Name: 8, dtype: object


In [37]:
theme_token = 'ECON_MONEYLAUNDERING'

manual_mask = df_head[8].str.split(';').apply(lambda toks: any(t == theme_token for t in toks) if isinstance(toks, list) else False)

print('Manual matches in head for', theme_token, ':', manual_mask.sum())

print(df_head[manual_mask].iloc[:3, :9])

Manual matches in head for ECON_MONEYLAUNDERING : 0
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []


In [42]:
summary = aml.filter_gkg_for_pld(delete_after_transform=False, output_mode='both', max_files=500)

print('Total matches:', summary['total_matches'])

TypeError: filter_gkg_for_pld() got an unexpected keyword argument 'max_files'